# Install packages

In [ ]:
!pip install transformers torch sentence-transformers

#Import Needed Packages

In [ ]:
from transformers import pipeline
from sentence_transformers import SentenceTransformer
from bertopic import BERTopic
from transformers import T5ForConditionalGeneration, T5Tokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation

# Sample Employee Feedback

In [ ]:
feedbacks =  [
    "The amount of work lately is too much; I need help to manage it.",
    "Working together as a team has boosted my motivation a lot.",
    "I often feel drained and lack support from colleagues.",
    "Flexible schedules make my work-life balance much better.",
    "The new projects are confusing and cause a lot of stress.",
    "I feel valued and my achievements are noticed.",
    "The increasing workload is pushing me towards burnout.",
    "Workshops and training sessions have really improved my skills.",
    "Management's communication is unclear and frustrating.",
    "I like my job, but meeting all the deadlines is challenging."
]

# 1. Sentiment Analysis

In [ ]:
sentiment_analyzer = pipeline("sentiment-analysis", model="distilbert-base-uncased-finetuned-sst-2-english")
sentiments = sentiment_analyzer(feedbacks)

Device set to use cuda:0


# 2. Stress / Burnout Detection

In [ ]:
# zero-shot classification for stress/burnout tagging
stress_classifier = pipeline("zero-shot-classification", model="facebook/bart-large-mnli")
stress_labels = ["Burnout", "Stress", "Satisfied", "Motivated"]
stress_results = [stress_classifier(fb, candidate_labels=stress_labels) for fb in feedbacks]

Device set to use cuda:0


# 3. Topic Extraction with LDA

In [ ]:
vectorizer = CountVectorizer(stop_words='english')
X = vectorizer.fit_transform(feedbacks)
lda = LatentDirichletAllocation(n_components=3, random_state=42)
lda.fit(X)
def display_topics(model, feature_names, no_top_words):
  topics = []
  for topic_idx, topic in enumerate(model.components_):
    top_words = [feature_names[i] for i in topic.argsort()[:-no_top_words - 1:-1]]
    topics.append(", ".join(top_words))
  return topics
topics = display_topics(lda, vectorizer.get_feature_names_out(), 5)

# 4. Summarization

In [ ]:
combined_feedback = " [SEP] ".join(feedbacks)
model_name = "t5-large"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)
prompt = "summarize: " + combined_feedback
inputs = tokenizer.encode(prompt, return_tensors="pt", max_length=512, truncation=True)
summary_ids = model.generate(
    inputs,
    max_length=150,
    min_length=50,
    length_penalty=2.0,
    num_beams=10,
    early_stopping=True,
    no_repeat_ngram_size=3
)
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)

In [ ]:
summary

'the amount of work lately is too much; i need help to manage it . flexible schedules make my work-life balance much better . workshops and training sessions have really improved my skills . meeting all the deadlines is challenging .'

# 5. HR Recommendation Tags

In [ ]:
recommendations = []
for sentiment, stress in zip(sentiments, stress_results):
  tags = []
  if "Burnout" in stress['labels'][:2] or "Stress" in stress['labels'][:2]:
    tags.append("Immediate Attention")
  elif sentiment['label'] == "NEGATIVE":
    tags.append("Monitor")
  else:
    tags.append("Positive Feedback")
  recommendations.append(tags)

# 6. Results

In [ ]:
for i, fb in enumerate(feedbacks):
  print(f"\nFeedback {i+1}: {fb}")
  print(f"Sentiment: {sentiments[i]}")
  print(f"Stress/Burnout Classification: {stress_results[i]['labels'][:3]}")
  print(f"HR Recommendation: {recommendations[i]}")

print("\nTop Topics:")
print(topics[0])
print("\nOverall Summary:")
print(summary)


Feedback 1: The amount of work lately is too much; I need help to manage it.
Sentiment: {'label': 'NEGATIVE', 'score': 0.9997158646583557}
Stress/Burnout Classification: ['Stress', 'Motivated', 'Burnout']
HR Recommendation: ['Immediate Attention']

Feedback 2: Working together as a team has boosted my motivation a lot.
Sentiment: {'label': 'POSITIVE', 'score': 0.9995494484901428}
Stress/Burnout Classification: ['Motivated', 'Satisfied', 'Stress']
HR Recommendation: ['Positive Feedback']

Feedback 3: I often feel drained and lack support from colleagues.
Sentiment: {'label': 'NEGATIVE', 'score': 0.9997932314872742}
Stress/Burnout Classification: ['Stress', 'Burnout', 'Motivated']
HR Recommendation: ['Immediate Attention']

Feedback 4: Flexible schedules make my work-life balance much better.
Sentiment: {'label': 'POSITIVE', 'score': 0.959521472454071}
Stress/Burnout Classification: ['Satisfied', 'Motivated', 'Stress']
HR Recommendation: ['Positive Feedback']

Feedback 5: The new projec